In [1]:
import pandas as pd

### Modify company name for your client here

In [4]:
df = pd.read_json('Metron.json')

In [5]:
df.head()

,body
0,"{""GroupDefinitions"":[{""Type"":""DIMENSION"",""Key""..."
1,"{""GroupDefinitions"":[{""Type"":""DIMENSION"",""Key""..."
2,"{""GroupDefinitions"":[{""Type"":""DIMENSION"",""Key""..."
3,"{""GroupDefinitions"":[{""Type"":""DIMENSION"",""Key""..."
4,"{""GroupDefinitions"":[{""Type"":""DIMENSION"",""Key""..."


In [6]:
dfs = [pd.read_json(df['body'][i], orient='index').transpose() for i in range(len(df))]

In [7]:
dfs[0]['ResultsByTime'][0].keys()

dict_keys(['TimePeriod', 'Total', 'Groups', 'Estimated'])

In [8]:
len(dfs)

385

In [9]:
def one_day_to_df_time(i):
    temp = dfs[i]['ResultsByTime'][0]
    start_0 = temp['TimePeriod']['Start']
    end_0 = temp['TimePeriod']['End']
    groups = temp['Groups']
    l = len(groups)
    start = []
    end = []
    for k in range(l):
        start.append(start_0)
        end.append(end_0)
    
    return (start,end)

In [10]:
def one_day_to_df_groups(i):
    temp = dfs[i]['ResultsByTime'][0]['Groups']
    l = len(temp)
    keys = []
    metrics = []
    for j in range(l):
        x = temp[j]['Keys']
        y = temp[j]['Metrics']
        keys.append(x)
        metrics.append(y)
    
    return (keys,metrics)

In [11]:
def one_day_to_df(i):
    (start,end) = one_day_to_df_time(i)
    (keys,metrics) = one_day_to_df_groups(i)
    df = pd.DataFrame({'Start_date':start,'End_date':end,'Service':keys,'Amount':metrics})
    
    return df

In [12]:
total_days = len(dfs)

In [13]:
df_list = []
for i in range(total_days):
    df_list.append(one_day_to_df(i))

In [14]:
df_final = pd.concat(df_list)

In [15]:
df_final = df_final.reset_index()

In [16]:
df_final = df_final.drop(['index'],axis=1)

In [17]:
df_final.head()

,Start_date,End_date,Service,Amount
0,2021-07-01,2021-07-02,[NoUsageType],"{'BlendedCost': {'Amount': '14897.04', 'Unit':..."
1,2021-07-01,2021-07-02,[AP-DataTransfer-Out-Bytes],"{'BlendedCost': {'Amount': '0.0387493548', 'Un..."
2,2021-07-01,2021-07-02,[AP-DataTransfer-Out-OBytes],"{'BlendedCost': {'Amount': '0.0010298178', 'Un..."
3,2021-07-01,2021-07-02,[AP-Requests-HTTP-Proxy],"{'BlendedCost': {'Amount': '0.0000675', 'Unit'..."
4,2021-07-01,2021-07-02,[AP-Requests-HTTPS-Proxy],"{'BlendedCost': {'Amount': '0.0320316', 'Unit'..."


In [18]:
df_final.Amount[0]['BlendedCost']['Unit']

'USD'

In [19]:
df_final.Amount[0]

{'BlendedCost': {'Amount': '14897.04', 'Unit': 'USD'},
 'UnblendedCost': {'Amount': '14897.04', 'Unit': 'USD'},
 'UsageQuantity': {'Amount': '0', 'Unit': ''}}

In [20]:
Currency = []
BlendedCost_Amount = []
UnblendedCost_Amount = []
UsageQuantity = []
    
for i in range(len(df_final)):
    temp = df_final.Amount[i]
    Currency.append(temp['BlendedCost']['Unit'])
    BlendedCost_Amount.append(temp['BlendedCost']['Amount'])
    UnblendedCost_Amount.append(temp['UnblendedCost']['Amount'])
    UsageQuantity.append(temp['UsageQuantity']['Amount'])
        
df_final['Currency']=Currency
df_final['BlendedCost_Amount']=BlendedCost_Amount
df_final['UnblendedCost_Amount']=UnblendedCost_Amount
df_final['UsageQuantity']=UsageQuantity

In [21]:
df_final = df_final.drop(['Amount'],axis=1)

In [22]:
df_final

,Start_date,End_date,Service,Currency,BlendedCost_Amount,UnblendedCost_Amount,UsageQuantity
0,2021-07-01,2021-07-02,[NoUsageType],USD,14897.04,14897.04,0
1,2021-07-01,2021-07-02,[AP-DataTransfer-Out-Bytes],USD,0.0387493548,0.0356494074,0.322911299
2,2021-07-01,2021-07-02,[AP-DataTransfer-Out-OBytes],USD,0.0010298178,0.0009474327,0.0171636317
3,2021-07-01,2021-07-02,[AP-Requests-HTTP-Proxy],USD,0.0000675,0.0000621,75
4,2021-07-01,2021-07-02,[AP-Requests-HTTPS-Proxy],USD,0.0320316,0.029469072,26693
...,...,...,...,...,...,...,...
177801,2022-07-20,2022-07-21,[eu-west-1-KMS-Requests],USD,0.149955,0.1379586,49985
177802,2022-07-20,2022-07-21,[eu-west-3-KMS-Keys],USD,3.443548332,3.1680644142,3.443548332
177803,2022-07-20,2022-07-21,[eu-west-3-KMS-Requests],USD,4.889427,4.49827284,1629809
177804,2022-07-20,2022-07-21,[sa-east-1-KMS-Keys],USD,0.479838702,0.4414515987,0.479838702


### Modify company name for your client here

In [23]:
df_final.to_excel('Metron_AWS_data_by_UsageType.xlsx', index=False)